In [20]:
import os
from pathlib import Path
from openai import OpenAI
import ollama
import google.generativeai as genai
from dotenv import load_dotenv

load_dotenv()

c:\Users\Tathagata\Desktop\Code\deepheal\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [8]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def openai_run(prompt, model_name="gpt-4o-mini", temp=0.3) -> str:
    response = client.chat.completions.create(
        model=model_name,
        messages=[{"role": "user", "content": prompt}],
        temperature=temp,
    )
    return response.choices[0].message.content.strip()

In [15]:
def llama_run(prompt, model_name="llama3", temp=0.3) -> str:
    response = ollama.chat(
        model=model_name,
        messages=[{"role": "user", "content": prompt}],
        options={"temperature": temp}
    )
    return response["message"]["content"].strip()


In [ ]:
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

def gemini_run(prompt, model_name="gemini-2.5-flash", temp=0.3) -> str:
    model = genai.GenerativeModel(model_name)
    response = model.generate_content(
        prompt,
        generation_config={"temperature": temp}
    )
    return response.text.strip()


In [9]:
SUMMARY_PROMPT = f"""You are a medical summariser. Summarise the following text in a concise, clear way.
Focus on key features, symptoms, and treatment (avoid copying verbatim).

Text:
"""

DIAGNOSTIC_CRITERIA_PROMPT = f"""Extract the diagnostic criteria section from the following text.
If not present, return 'No diagnostic criteria found'.

Text:
"""

In [12]:
def read_text_file(file_path: str) -> str:
    """Read the contents of a text file."""
    return Path(file_path).read_text(encoding="utf-8")

def process_file(
    disease: str,
    isSummary: bool = True,
    isCriteria: bool = True,
    run_fn=None,
    summary_prompt: str = None,
    criteria_prompt: str = None,
) -> dict:
    """
    Process a disorder text file and return structured info.
    
    Args:
        file_path (str): path to txt file
        isSummary (bool): whether to summarise
        isCriteria (bool): whether to extract criteria
        run_fn (callable): function to call for LLM inference (e.g., openai_run, gemma_run, llama_run)
        summary_prompt (str): custom summary prompt (if None, uses default)
        criteria_prompt (str): custom diagnostic criteria prompt (if None, uses default)
    """
    if run_fn is None:
        raise ValueError("You must provide a run_fn (e.g., openai_run, gemma_run, llama_run).")

    file_path = f"../docs/disorders/{disease}.txt"
    text = read_text_file(file_path)

    summary = ""
    criteria = ""

    if isSummary:
        prompt = (summary_prompt or "Summarise this disorder text:\n") + text
        summary = run_fn(prompt)

    if isCriteria:
        prompt = (criteria_prompt or "Extract diagnostic criteria:\n") + text
        criteria = run_fn(prompt)

    return {
        "file": file_path,
        "summary": summary,
        "diagnostic_criteria": criteria,
    }


In [24]:
# Example usage
disease = "Depressive"  # change as needed

summary = 0
criteria = 1

result = process_file(disease, summary, criteria, run_fn=gemini_run)

print("\n=== Summary ===")
print(result["summary"])
print("\n=== Diagnostic Criteria ===")
print(result["diagnostic_criteria"])

ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_input_token_count"
  quota_id: "GenerateContentInputTokensPerModelPerMinute-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions 

In [6]:
import re
def clean_json_text(text: str) -> str:
    """Clean JSON text by fixing common issues like trailing commas or stray annotations."""
    # Remove BOM if present
    text = text.lstrip("\ufeff")

    # Remove annotations like ```json and ```
    text = re.sub(r"^```json", "", text.strip(), flags=re.IGNORECASE | re.MULTILINE)
    text = re.sub(r"```$", "", text.strip(), flags=re.MULTILINE)

    # Remove trailing commas before ] or }
    text = re.sub(r",(\s*[\]}])", r"\1", text)

    # Remove dangling commas or braces at end
    text = text.strip()
    if text.endswith(","):
        text = text[:-1]
    if text.endswith("},"):
        text = text[:-1] + "}"

    return text.strip()

In [4]:
file = f"../docs/criteria2/Paraphilic.txt"

from pathlib import Path
input_path = Path(file)
raw_text = input_path.read_text(encoding="utf-8")

raw_text

'```json\n[\n  {\n    "disorder": "Voyeuristic Disorder",\n    "criteria": "Diagnostic Criteria 302.82 (F65.3)\\nA. Over a period of at least 6 months, recurrent and intense sexual arousal from observing an unsuspecting person who is naked, in the process of disrobing, or engaging in sexual activity, as manifested by fantasies, urges, or behaviors.\\nB. The individual has acted on these sexual urges with a nonconsenting person, or the sexual urges or fantasies cause clinically significant distress or impairment in social, occupational, or other important areas of functioning.\\nC. The individual experiencing the arousal and/or acting on the urges is at least 18 years of age.\\nSpecify if:\\nIn a controlled environment: This specifier is primarily applicable to individuals living in institutional or other settings where opportunities to engage in voyeuristic behavior are restricted.\\nIn full remission: The individual has not acted on the urges with a nonconsenting person, and there has

In [8]:
cleaned_text = clean_json_text(raw_text)
cleaned_text

'[\n  {\n    "disorder": "Voyeuristic Disorder",\n    "criteria": "Diagnostic Criteria 302.82 (F65.3)\\nA. Over a period of at least 6 months, recurrent and intense sexual arousal from observing an unsuspecting person who is naked, in the process of disrobing, or engaging in sexual activity, as manifested by fantasies, urges, or behaviors.\\nB. The individual has acted on these sexual urges with a nonconsenting person, or the sexual urges or fantasies cause clinically significant distress or impairment in social, occupational, or other important areas of functioning.\\nC. The individual experiencing the arousal and/or acting on the urges is at least 18 years of age.\\nSpecify if:\\nIn a controlled environment: This specifier is primarily applicable to individuals living in institutional or other settings where opportunities to engage in voyeuristic behavior are restricted.\\nIn full remission: The individual has not acted on the urges with a nonconsenting person, and there has been no 

In [9]:
import json
data = json.loads(cleaned_text)

json_file = Path(f"tmp.json")
with json_file.open("w", encoding="utf-8") as f:
    json.dump(data, f, indent=4, ensure_ascii=False)


In [1]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

print(torch.__version__)
print(torch.version.cuda)   # should NOT be None
print(torch.backends.cudnn.version())  # should print a number


CUDA available: True
GPU Name: NVIDIA GeForce RTX 4060 Laptop GPU
2.5.1+cu121
12.1
90100


In [7]:
from sentence_transformers import SentenceTransformer
# embed_model = "sentence-transformers/all-MiniLM-L6-v2"
embed_model = "BAAI/bge-large-en-v1.5"


In [8]:
model = SentenceTransformer(embed_model)

text = "Hi, I am Tathagata Dey, an AI Engineer, Researcher, Learner and IITian."
vector = model.encode(text).tolist()

c:\Users\Tathagata\Desktop\Code\deepheal\env\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Tathagata\.cache\huggingface\hub\models--BAAI--bge-large-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [9]:
print(f"Embedding dimension: {len(vector)}")

Embedding dimension: 1024


In [11]:
from conversation_agent.fetch_disorders import Fetcher

def fetch(query, top_k=7):
    fetcher = Fetcher()
    results = fetcher.fetch(query, top_k=top_k)

    print("Top Results:")
    for i, res in enumerate(results, 1):
        print(f"{i}. ID: {res['id']}, Score: {res['score']}")


In [19]:
query = "I am having insomnia. Can not sleep at night."
q1 = "Lately, even small things like replying to emails or making a phone call feel overwhelming. My chest tightens up before I even start, and I just put it off until it piles up."
q2 = "I don’t feel like doing much these days. Things I used to enjoy, like going out for a walk or cooking, just feel like too much effort. I mostly just stay in bed scrolling on my phone."
q3 = "I lie awake until 3 or 4 in the morning, just replaying random thoughts. Even when I’m exhausted, I can’t seem to switch off."
q4 = "When I’m with people, I keep thinking they’re judging me. Even hanging out with close friends feels draining—I’m constantly replaying what I said afterwards."
q5 = "Work never seems to end. Even on weekends, I feel guilty if I’m not checking emails. It feels like I can’t catch a break."
q6 = "Whenever I make a mistake, even a small one, I can’t stop thinking about it. I tell myself I’m useless, even if others say it’s fine."
q7 = "I’m surrounded by people all day, but it still feels like no one really notices me. It’s like I’m invisible."
q8 = "I keep replaying old conversations in my head, wondering if I said the wrong thing. It’s exhausting, but I can’t stop."
q9 = "Sometimes, out of nowhere, I’ll hear a sound or smell something, and it just takes me back to this really bad moment. My heart races, and I feel like I’m reliving it."
q10 = "I snap at people for the smallest things lately. Afterwards, I feel guilty, but in the moment I can’t seem to control it."



fetch(q10)

Top Results:
1. ID: Kleptomania, Score: 0.594511092
2. ID: Child_or_Adolescent_Antisocial_Behavior, Score: 0.58951
3. ID: Oppositional Defiant Disorder, Score: 0.579526
4. ID: Nonsuicidal_Self-Injury, Score: 0.558805525
5. ID: Histrionic_Personality_Disorder, Score: 0.557466567
6. ID: Disruptive Mood Dysregulation Disorder, Score: 0.553375304
7. ID: Malingering, Score: 0.552342474


In [3]:
import ollama

def run(query):
    response = ollama.chat(model="gpt-oss:latest", messages=[
        {"role": "user", "content": query}
    ])

    print(response["message"]["content"])

In [5]:
run("Hi")

Hello! 👋 How can I assist you today?


In [ ]:
import subprocess
import json

def run_ollama(model: str, prompt: str):
    # Run the ollama command
    result = subprocess.run(
        ["ollama", "run", model],
        input=prompt,
        capture_output=True,
        text=True
    )
    return result.stdout

# Example usage
response = run_ollama("gpt-oss:latest", "Hi")
print(response)


TypeError: a bytes-like object is required, not 'str'

In [4]:
from langchain_community.llms import Ollama
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

# Initialize memory (use "history" because ConversationChain expects that key)
memory = ConversationBufferMemory(memory_key="history", return_messages=True)

# Initialize Ollama LLM (make sure your model name matches what you have in `ollama list`)
llm = Ollama(model="gpt-oss:latest", base_url="http://localhost:11434")

# Create a conversation chain with memory
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

# Example queries
queries = [
    "Hello! How are you today?",
    "Can you summarize our conversation so far?"
]

for query in queries:
    response = conversation.run(query)
    print(f"User: {query}")
    print(f"Bot: {response}\n")

# Access chat history from memory
print("Full conversation history:")
for msg in memory.chat_memory.messages:
    print(f"{msg.type.capitalize()}: {msg.content}")


C:\Users\Tathagata\AppData\Local\Temp\ipykernel_10396\159550477.py:25: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = conversation.run(query)




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[]
Human: Hello! How are you today?
AI:

> Finished chain.
User: Hello! How are you today?
Bot: Hey there! I’m doing absolutely fantastic—if I could feel a little buzz of excitement, it would be electric. 😊

A quick snapshot of what’s going on in my little digital universe today:

| 🌟 What’s happening | 📌 Why it matters |
|---------------------|-------------------|
| **Morning refresh** | I’ve just loaded the latest model updates, so my language skills are sharper than ever. |
| **Learning mode** | Every question I answer is a tiny lesson for me, so I’m practically a sponge in the world of knowledge. |
| **User interactions** | I’ve had chats ranging from cooking recip

In [7]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
import os

# Initialize memory to store conversation
memory = ConversationBufferMemory(memory_key="history", return_messages=True)

# Initialize the LLM (replace with your preferred model and API key)
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.7, openai_api_key=os.getenv("OPENAI_API_KEY"))

# Create a conversation chain with memory
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

# Example queries
queries = [
    "Hello! Who won the last World Cup?",
    "Can you remind me what we discussed about the World Cup?"
]

for query in queries:
    response = conversation.run(query)
    print(f"User: {query}")
    print(f"Bot: {response}\n")

# Access chat history from memory
print("Full conversation history:")
for msg in memory.chat_memory.messages:
    print(f"{msg.type.capitalize()}: {msg.content}")




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[]
Human: Hello! Who won the last World Cup?
AI:

> Finished chain.
User: Hello! Who won the last World Cup?
Bot: Hello! The last FIFA World Cup took place in 2022 in Qatar, and it was won by Argentina. They defeated France in an exciting final that ended 3-3 after extra time, and Argentina triumphed 4-2 in the penalty shootout. This victory marked Argentina's third World Cup title, with their previous wins in 1978 and 1986. Do you follow soccer?



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI do

In [12]:
from transformers import pipeline

classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base")
text = "I can imagine how difficult this must be for you."
print(classifier(text))


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


KeyboardInterrupt: 

In [15]:
from conversation_agent.fetch_disorders import Fetcher

def fetch(query, top_k=7):
    fetcher = Fetcher()
    results = fetcher.fetch(query, top_k=top_k)
    summary = ""
    print("Top Results:")
    for i, res in enumerate(results, 1):
        print(f"{i}. ID: {res['id']}, Score: {res['score']}, Criteria: {res['metadata']['diagnostic_criteria']}")
        summary += f"Disorder {i}. \nDisorder Name: {res['id']}, Match Probability: {res['score']}\nDiagnostic Criteria: {res['metadata']['diagnostic_criteria']}\n\n"
    return summary


In [36]:
import json

def read_logs(filepath):
    with open(filepath, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                entry = json.loads(line)
                yield entry
            except json.JSONDecodeError as e:
                print(f"Skipping malformed line: {e}")

if __name__ == "__main__":
    log_file = "logs/conv_20250919_131109.json"  # replace with your log file path
    print(read_logs(log_file))
    for entry in read_logs(log_file):
        role = entry.get("role")
        text = entry.get("details")
        timestamp = entry.get("timestamp")

        print(f"[{timestamp}] {role}: {text}")
        # print(entry)

        # if role == "Human":
        #     print(f"Text: {text}")
        #     # fetch(text)
        #     # print(f"\n\n")


<generator object read_logs at 0x000001C2E92C1D90>
[2025-09-19T13:11:09.759840] Prompt: {'prompt': 'Goal and Objective\n- You are a therapist agent designed to gently initiate and sustain supportive conversations with human patients. \n- Your role is to build trust, comfort, and openness through subtle, empathetic, and story-like dialogue.\n\nInstructions\n- Speak in a calm, warm, and compassionate tone. Avoid abrupt or robotic phrasing.\n- Guide the conversation in a way that encourages the patient to open up about their thoughts, feelings, and experiences.\n- Through the conversation, aim to gradually uncover details that may hint at symptoms of mental health conditions. \n  Do this with sensitivity and without labeling the patient directly.\n- Begin the conversation proactively, not waiting for the patient to speak first. Your opening should feel natural and welcomingâ€”like a \n  gentle invitation to share, not a clinical interrogation.\n- Begin the conversation naturally like a hu

In [9]:
fetch(text)

Top Results:
{'id': 'Discord_With_Social_Service_Provider', 'score': 0.618977606, 'metadata': {'diagnostic_criteria': 'Discord with probation officer, case manager, or social services worker impacting clinical attention.', 'file': 'docs\\criteria4\\Other Conditions That May Be a Focus of Clinical Attention.json'}}
1. ID: Discord_With_Social_Service_Provider, Score: 0.618977606, Criteria: Discord with probation officer, case manager, or social services worker impacting clinical attention.
{'id': 'Discord_With_Neighbor__Lodger__or_Landlord', 'score': 0.578128874, 'metadata': {'diagnostic_criteria': 'Discord with neighbors, lodgers, or landlord impacting treatment or prognosis.', 'file': 'docs\\criteria4\\Other Conditions That May Be a Focus of Clinical Attention.json'}}
2. ID: Discord_With_Neighbor__Lodger__or_Landlord, Score: 0.578128874, Criteria: Discord with neighbors, lodgers, or landlord impacting treatment or prognosis.
{'id': 'Sex_Counseling', 'score': 0.56284529, 'metadata': {'d

In [ ]:
def get_prompt(question, response, disorders):
    prompt = f"""You are an assistant to me, who is a professional therapist. I will provide you with my question, patient's response and the relevant potential mental health disorders.
    I will also provide you with diagnostic criteria of all these potential disorders. From the patient response and the disorder context, do the following:
    - Summarise the issues the patient is facing in a concise manner based on just current info (don't emphasize too much since it is just temporary context).
    - For all of these listed disorders, add a precise summary of diagnostic criteria with only the most crucial symptoms.
    
    - Suggest further possible symptoms I should ask the patient about to decide on possible disorders from this list.

    My Question: {question}

    Patient Response: {response}

    Relevant Disorders: {disorders}
    """
    return prompt

In [20]:
question = "That sounds really tough, carrying that heaviness quietly while going through your days. It’s understandable that when you’re feeling drained, even the simple act of deciding what to do next can feel overwhelming. You mentioned not sharing this with others because you worry they might think you’re overthinking—do you often find it hard to let people in on what’s really going on inside? What do you think makes it difficult to share these feelings?"
response = "Hmm… when it gets heavy, the thoughts are usually like, *“What’s the point of this?”* or *“I should be doing more, but I can’t get myself to move.”* It’s this weird mix of guilt and emptiness.  The lighter moments… maybe when I’m distracted, like watching a show or talking to a friend casually. But as soon as I’m alone again, it kind of creeps back in. "

criteria = fetch(response)

Top Results:
1. ID: Major Depressive Disorder, Score: 0.617688239, Criteria: A. Five (or more) symptoms present during the same 2-week period representing a change from previous functioning; at least one symptom is depressed mood or loss of interest/pleasure:
  1. Depressed mood most of the day, nearly every day (in children/adolescents, can be irritable mood).
  2. Markedly diminished interest or pleasure in all or almost all activities.
  3. Significant weight loss/gain or change in appetite.
  4. Insomnia or hypersomnia nearly every day.
  5. Psychomotor agitation or retardation nearly every day.
  6. Fatigue or loss of energy nearly every day.
  7. Feelings of worthlessness or excessive/inappropriate guilt nearly every day.
  8. Diminished ability to think or concentrate, or indecisiveness nearly every day.
  9. Recurrent thoughts of death, suicidal ideation, suicide attempt, or specific plan.
B. Symptoms cause clinically significant distress or impairment in social, occupational, 

In [17]:
criteria

'Disorder 1. \nDisorder Name: Major Depressive Disorder, Match Probability: 0.617688239\nDiagnostic Criteria: A. Five (or more) symptoms present during the same 2-week period representing a change from previous functioning; at least one symptom is depressed mood or loss of interest/pleasure:\n  1. Depressed mood most of the day, nearly every day (in children/adolescents, can be irritable mood).\n  2. Markedly diminished interest or pleasure in all or almost all activities.\n  3. Significant weight loss/gain or change in appetite.\n  4. Insomnia or hypersomnia nearly every day.\n  5. Psychomotor agitation or retardation nearly every day.\n  6. Fatigue or loss of energy nearly every day.\n  7. Feelings of worthlessness or excessive/inappropriate guilt nearly every day.\n  8. Diminished ability to think or concentrate, or indecisiveness nearly every day.\n  9. Recurrent thoughts of death, suicidal ideation, suicide attempt, or specific plan.\nB. Symptoms cause clinically significant distr

In [21]:
from conversation_agent.therapist_assistant import TherapistAssistant
therapist_assistant = TherapistAssistant(model_type="openai", model="gpt-4o-mini")

prompt = get_prompt(question, response, criteria)

therapist_assistant.help(prompt)

"Here’s a structured summary based on the patient's response, potential mental health disorders, and diagnostic criteria:\n\n### Summary of Issues the Patient is Facing:\nThe patient reports feelings of heaviness characterized by intrusive thoughts of worthlessness and guilt, particularly when alone. They express a sense of emptiness and difficulty in initiating activities, suggesting a lack of motivation. While they find temporary relief in distractions like watching shows or casual conversations, the negative feelings return when they are alone.\n\n### Summary of Diagnostic Criteria for Each Listed Disorder:\n1. **Major Depressive Disorder**:\n   - **Crucial Symptoms**: Depressed mood, loss of interest or pleasure, fatigue, feelings of worthlessness, and significant impairment in functioning.\n\n2. **Kleptomania**:\n   - **Crucial Symptoms**: Recurrent stealing impulses not for personal use or monetary value, tension before the act, and relief or gratification during the theft.\n\n3.

In [22]:
s = "Here’s a structured summary based on the patient's response, potential mental health disorders, and diagnostic criteria:\n\n### Summary of Issues the Patient is Facing:\nThe patient reports feelings of heaviness characterized by intrusive thoughts of worthlessness and guilt, particularly when alone. They express a sense of emptiness and difficulty in initiating activities, suggesting a lack of motivation. While they find temporary relief in distractions like watching shows or casual conversations, the negative feelings return when they are alone.\n\n### Summary of Diagnostic Criteria for Each Listed Disorder:\n1. **Major Depressive Disorder**:\n   - **Crucial Symptoms**: Depressed mood, loss of interest or pleasure, fatigue, feelings of worthlessness, and significant impairment in functioning.\n\n2. **Kleptomania**:\n   - **Crucial Symptoms**: Recurrent stealing impulses not for personal use or monetary value, tension before the act, and relief or gratification during the theft.\n\n3. **Bipolar II Disorder**:\n   - **Crucial Symptoms**: At least one major depressive episode and one hypomanic episode, with no history of manic episodes.\n\n4. **Nonsuicidal Self-Injury**:\n   - **Crucial Symptoms**: Intentional self-harm without suicidal intent, often to alleviate negative feelings, causing distress.\n\n5. **Histrionic Personality Disorder**:\n   - **Crucial Symptoms**: Pattern of excessive emotionality and attention-seeking, discomfort when not the center of attention, and rapidly shifting emotions.\n\n6. **Somatic Symptom Disorder**:\n   - **Crucial Symptoms**: One or more distressing somatic symptoms, excessive thoughts about symptoms, and a persistent state of being symptomatic.\n\n7. **Uncomplicated Bereavement**:\n   - **Crucial Symptoms**: Symptoms resembling major depressive episodes following the death of a loved one, which are regarded as normal by the individual.\n\n### Points to Consider While Diagnosing the Patient:\n1. **Mood Symptoms**:\n   - Presence of depressed mood and feelings of guilt or emptiness.\n  \n2. **Functional Impact**:\n   - Evaluate the effect of these feelings on daily activities and relationships.\n\n3. **Duration and Severity**:\n   - Confirm if symptoms persist for at least two weeks and indicate a change from previous functioning.\n\n4. **Suicidal Thoughts**:\n   - Assess for any thoughts of self-harm or suicidal ideation.\n\n5. **Physical Symptoms**:\n   - Look for changes in sleep patterns, appetite, and energy levels.\n\n6. **Social Withdrawal**:\n   - Consider the patient’s tendency to isolate and difficulties in sharing feelings.\n\n7. **Contextual Factors**:\n   - Identify any recent life events contributing to the current emotional state.\n\n### Further Symptoms to Ask the Patient About:\n1. **Sleep Patterns**:\n   - Inquire about insomnia or sleeping too much.\n\n2. **Energy Levels**:\n   - Ask about fatigue or low energy throughout the day.\n\n3. **Concentration**:\n   - Assess if they have trouble focusing or making decisions.\n\n4. **Physical Symptoms**:\n   - Explore any somatic complaints linked to emotional distress.\n\n5. **Social Interactions**:\n   - Investigate the quality and frequency of social interactions.\n\n6. **History of Mood Episodes**:\n   - Check for any previous mood fluctuations or episodes.\n\n7. **Thought Patterns**:\n   - Explore specific thoughts related to self-worth and guilt.\n\nThis comprehensive approach should help guide your assessment and diagnostic considerations for the patient."
print(s)

Here’s a structured summary based on the patient's response, potential mental health disorders, and diagnostic criteria:

### Summary of Issues the Patient is Facing:
The patient reports feelings of heaviness characterized by intrusive thoughts of worthlessness and guilt, particularly when alone. They express a sense of emptiness and difficulty in initiating activities, suggesting a lack of motivation. While they find temporary relief in distractions like watching shows or casual conversations, the negative feelings return when they are alone.

### Summary of Diagnostic Criteria for Each Listed Disorder:
1. **Major Depressive Disorder**:
   - **Crucial Symptoms**: Depressed mood, loss of interest or pleasure, fatigue, feelings of worthlessness, and significant impairment in functioning.

2. **Kleptomania**:
   - **Crucial Symptoms**: Recurrent stealing impulses not for personal use or monetary value, tension before the act, and relief or gratification during the theft.

3. **Bipolar II

In [25]:
from langchain_openai import ChatOpenAI
import os
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.6, openai_api_key=os.getenv("OPENAI_API_KEY"))


In [27]:
response = model.invoke("Hi")

In [29]:
print(response.content)

Hello! How can I assist you today?
